In [1]:
import os
import hddm
import pickle
from kabuki.analyze import gelman_rubin
import kabuki
from IPython.parallel import Client

/Users/minheeyoo/opt/anaconda3/envs/hddmenv/lib/python3.6/site-packages/IPython/parallel.py:13: ShimWarning: The `IPython.parallel` package has been deprecated since IPython 4.0. You should import from ipyparallel instead.
  "You should import from ipyparallel instead.", ShimWarning)


In [2]:
data = hddm.load_csv('CavanaghDataset_full_hddm.csv')
data

,subj_idx,response,rt,x1,x2,x3,x4,x5,x6
0,1,0,0.562,0,0,0,-1,0,1
1,1,1,0.456,-1,0,0,0,1,0
2,1,0,0.689,0,-1,1,0,0,0
3,1,1,0.491,-1,0,0,1,0,0
4,1,1,0.435,-1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...
4661,1,1,0.548,0,0,0,-1,0,1
4662,1,1,0.619,0,-1,0,0,1,0
4663,1,1,0.598,-1,1,0,0,0,0
4664,1,1,0.530,0,0,0,0,-1,1


In [3]:
def run_model(i):
    import os
    import hddm
    import pickle
    pathAnalysis = '/project/Cavanagh'
    os.chdir(pathAnalysis)
    data = hddm.load_csv(pathAnalysis + '/' + 'CavanaghDataset_full_hddm.csv')
    # Fit model
    v_reg = {'model': 'v ~ 0+x1+x2+x3+x4+x5+x6', 
             'link_func': lambda x: x}
    reg_descr = [v_reg]
    m = hddm.HDDMRegressor(data, reg_descr, 
                           group_only_regressors=False, 
                           p_outlier = 0.0)
    m.sample(10000, burn=4000, dbname='Cavanagh_chain%i.db'%i, db='pickle')
    return m

In [4]:
v = Client()[:]
jobs = v.map(run_model, range(4)) # 4 is the number of CPUs
model_set = jobs.get()

In [5]:
# save model set
with open('Cavanagh_hddm_models', 'wb') as f:
    pickle.dump(model_set, f)

In [6]:
models = kabuki.utils.concat_models(model_set)

In [8]:
# save stats
path = '/project/Cavanagh'
# save samples
samples = models.get_traces()
samples.to_csv(path + '/'+'Cavanagh_samples.csv')